### Setup Instructions

If you haven't done so, please follow the [setup instructions](./nlp-tutorial-setup) to prepare your environment.  This tutorial will reference the python script [nlpflow.py](https://github.com/outerbounds/tutorials/blob/main/nlp/nlpflow.py).

#### What You Will Learn
At the end of this lesson, you will:
    
* Learn how to collaborate on and organize flows with tagging.
* Learn design patterns for testing your models.

In Lesson 4, you saw how we trained a model and compared it to a baseline.  However, what if your model is worse than the baseline?  Is there a way to manage this situation programmatically? An important Metaflow feature that can enable this is [Tagging](https://outerbounds.com/blog/five-ways-to-use-the-new-metaflow-tags).  Tagging allows you to categorize and organize flows, which we can use to mark certain models as "production candidates.”

Tags allow you to express opinions about results of your and your colleagues' work, and, importantly, change those assessments at any time. In contrast to runs and artifacts that represent immutable facts (history shouldn't be rewritten), the way how you interpret those facts may change over time, which is reflected in tags.  This makes tags ideal for managing which models are promoted to the next step in your modeling workflow.

You can add a tag to a flow with only a few lines of code.  Below is a snippet of code we will use to add tags in our flow:

```python
from metaflow import Flow, current
run = Flow(current.flow_name)[current.run_id]
run.add_tag('deployment_candidate')
```

In this flow, we modify our `end` step to apply the tag `deployment_candidate` if our model passes two tests: (1) a baseline (2) and a smoke test.

Concretely, we will add the following to the `end` step:

1. **A smoke test** that tests that the model is performing correctly against very easy examples that it should not be getting wrong.  A smoke test is a lightweight way to catch unexpected behaviors in your model, even if your model is beating the baseline. 
2. **A comparison of the model with the baseline**. We are going to check if our model's AUC score is better than the baseline.  There are more advanced variations on this technique, including using other models for baselines, or requiring that your model performs better than the baseline by a specific margin.  We leave these variations as an excercise for the reader.
3. **Add a tag** if our model passes the smoke test and beats the baseline.

In [ ]:
%%writefile nlpflow.py

from metaflow import FlowSpec, step, Flow, current

class NLPFlow(FlowSpec):
        
    @step
    def start(self):
        "Read the data"
        import pandas as pd
        self.df = pd.read_parquet('train.parquet')
        self.valdf = pd.read_parquet('valid.parquet')
        print(f'num of rows: {self.df.shape[0]}')
        self.next(self.baseline, self.train)

    @step
    def baseline(self):
        "Compute the baseline"
        from sklearn.metrics import accuracy_score, roc_auc_score
        baseline_predictions = [1] * self.valdf.shape[0]
        self.base_acc = accuracy_score(self.valdf.labels, baseline_predictions)
        self.base_rocauc = roc_auc_score(self.valdf.labels, baseline_predictions)
        self.next(self.join)

    @step
    def train(self):
        "Train the model"
        from model import NbowModel
        model = NbowModel(vocab_sz=750)
        model.fit(X=self.df['review'], y=self.df['labels'])
        self.model_dict = model.model_dict #save model
        self.next(self.join)
        
    @step
    def join(self, inputs):
        "Compare the model results with the baseline."
        import pandas as pd
        from model import NbowModel
        self.model_dict = inputs.train.model_dict
        self.train_df = inputs.train.df
        self.val_df = inputs.baseline.valdf
        self.base_rocauc = inputs.baseline.base_rocauc
        self.base_acc = inputs.baseline.base_acc
        model = NbowModel.from_dict(self.model_dict)
        
        self.model_acc = model.eval_acc(X=self.val_df['review'], labels=self.val_df['labels'])
        self.model_rocauc = model.eval_rocauc(X=self.val_df['review'], labels=self.val_df['labels'])
        
        print(f'Baseline Acccuracy: {self.base_acc:.2%}')
        print(f'Baseline AUC: {self.base_rocauc:.2}')
        print(f'Model Acccuracy: {self.model_acc:.2%}')
        print(f'Model AUC: {self.model_rocauc:.2}')
        self.next(self.end)
        
    @step
    def end(self):
        "Tags model as a deployment candidate if it beats the baseline and passes smoke tests."
        from model import NbowModel
        model = NbowModel.from_dict(self.model_dict)
        
        self.beats_baseline = self.model_rocauc > self.base_rocauc
        print(f'Model beats baseline (T/F): {self.beats_baseline}')
        #smoke test to make sure model is doing the right thing on obvious examples.
        _tst_reviews = ["poor fit its baggy in places where it isn't supposed to be.",
                        "love it, very high quality and great value"]
        _tst_preds = model.predict(_tst_reviews)
        self.passed_smoke_test = _tst_preds[0][0] < .5 and _tst_preds[1][0] > .5
        print(f'Model passed smoke test (T/F): {self.passed_smoke_test}')
        
        if self.beats_baseline and self.passed_smoke_test:
            run = Flow(current.flow_name)[current.run_id]
            run.add_tag('deployment_candidate')
        

if __name__ == '__main__':
    NLPFlow()

Overwriting nlpflow.py


In [ ]:
#notest
! python nlpflow.py run

Metaflow 2.7.1 executing NLPFlow for user:hamel
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-08-11 11:05:39.966 Workflow starting (run-id 1660241139962368):
2022-08-11 11:05:39.976 [1660241139962368/start/1 (pid 31012)] Task is starting.
2022-08-11 11:05:40.887 [1660241139962368/start/1 (pid 31012)] num of rows: 20377
2022-08-11 11:05:40.994 [1660241139962368/start/1 (pid 31012)] Task finished successfully.
2022-08-11 11:05:41.005 [1660241139962368/baseline/2 (pid 31016)] Task is starting.
2022-08-11 11:05:41.015 [1660241139962368/train/3 (pid 31017)] Task is starting.
2022-08-11 11:05:42.300 [1660241139962368/baseline/2 (pid 31016)] Task finished successfully.
2022-08-11 11:05:43.527 [1660241139962368/train/3 (pid 31017)] 2022-08-11 11:05:43.527783: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance

### Next Steps

Now that we have tagged our model if it meets our minimum standards, we are now ready to use this model in downstream workflows.  In the next lesson, we will explore different ways you can utilize the model you have trained.